In [2]:
import json

In [3]:
har_path = '.../data/har/learn.pwskills.com_Archive [23-02-27 18-38-31] - QuizPage.har'
har_path = '.../data/har/learn.pwskills.com_Archive [23-02-26 14-41-19].har'
har_path = '.../data/har/learn.pwskills.com_Archive [23-02-28 20-46-43] - MainPage.har'

data: list[dict]= json.load(open(har_path))['log']['entries']

# HAR files analysis

## What are HAR files?

▶️ **HAR** (HTTP Archive) is a file format used to store HTTP interaction data between a web browser and a web server. The data typically includes information such as HTTP requests, responses, headers, timings, and more.

▶️ **HAR** files are commonly used for debugging and analyzing web performance issues, as they provide detailed insights into the various network requests made by a webpage and their corresponding timings.

▶️ They can also be used for security auditing and testing, as they can reveal potential security vulnerabilities and sensitive information leakage.

In [4]:
api_bindings_set = set()
api_bindings_list = list()

for i in data:
    if 'pwskills' in  (url := i['request']['url']) and i['request']['method'] == 'GET':
        if 'application/json' in i['request']['headers'][2]['value']:
            api_bindings_set.update(url[8:].split('/'))
            api_bindings_list.append(url[8:].split('/'))
            print(url)


https://api.pwskills.com/v1/learn/user/isCourseBought/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/auth/profile
https://api.pwskills.com/v1/learn/section/currentLiveClass/course/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/course/63a2eb428899436daf7eb489?withAllCourseMetas=true&ignoreInActive=true
https://api.pwskills.com/v1/learn/submission/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/learn/lesson/course/63a2eb428899436daf7eb489/63fb5e2d182c674c454b9aa8
https://api.pwskills.com/v1/learn/lesson/video-session/63a2eb428899436daf7eb489/lesson/63fb5e2d182c674c454b9aa8
https://api.pwskills.com/v1/learn/analytics/progress/course/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/learn/lesson/course/63a2eb428899436daf7eb489/63fb7e48182c67a2a64b9b0a


## Conclusion 1

1. All the URLs have `'https://api.pwskills.com/v1/'`. It is like root/base URL for the api requests.

    ```json
    // https://api.pwskills.com/v1?all=true

    {
      "env": "production",
      "instance": "0.8391243622921039",
      "version": "1.83.4",
      "all": {
        "totalmem": 32.823218176,
        "freemem": 29.265063936
      }
    }
    ```

2. All the above URLs are `authorized`. These URL's respone are take using `cURL` command by providing the header `Authorization`.  
_Authorization requires a Bearer._

    ```json
    { "error": "Unauthorized" }
    ```

3. [URL](https://api.pwskills.com/v1/course/63a2eb428899436daf7eb489?withAllCourseMetas=true&ignoreInActive=true) is public to use, it doesn't require any authorization for GET request.  
This gives all the data related to the specified course. _Courses are identify using its Course ID_.  

> Data Science Masters Course ID: `63a2eb428899436daf7eb489`


# Authorized URLs

A fuction which takes `Request Headers` and takes `list[dict]` as argument and `return bool` if it has Authorization in it.

In [18]:
def check_auth_request(headers: list[dict]) -> bool:
    for head in headers:
        if 'Authorization' in head.values():
            return True
    return False

In [19]:
for i in data:
    if 'pwskills' in  (url := i['request']['url']):
        if check_auth_request(i['request']['headers']):
            print(url)

https://api.pwskills.com/v1/learn/analytics/lastPlayed/course/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/auth/profile
https://api.pwskills.com/v1/learn/user/isCourseBought/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/learn/section/currentLiveClass/course/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/learn/submission/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/learn/lesson/video-session/63a2eb428899436daf7eb489/lesson/63fa15eb182c67f36e4b90dd
https://api.pwskills.com/v1/learn/lesson/course/63a2eb428899436daf7eb489/63fa15eb182c67f36e4b90dd
https://api.pwskills.com/v1/course/63a2eb428899436daf7eb489?withAllCourseMetas=true&ignoreInActive=true
https://api.pwskills.com/v1/learn/analytics/progress/course/63a2eb428899436daf7eb489
https://api.pwskills.com/v1/learn/user/recentlyAccessedCourses


## How to get the quizes data.

1. You can get it by using `cURL` command and by providing `Authorization` request header.
2. Try to create a session using `requests` library and scrape the data.
3. Web scrapping using selenium.

### Problems for fetching the data <small><sub>respectively</sub></small>

1. The `Authorization` IDs are changes for every request.
2. If use `requests` library for Session State you have to login with `Google`. Means you cannot provide __payload__ of _email and password_.
3. Same goes with `Selenium` method. It also require the Google SignIn part to get into the website.

> Also this applies to all the _Authorized URLs_.

# URL Analysis

## Web Surfline

```mermaid
flowchart
    home{Home Page}
    video{{Watch Course Video}}
    quiz[Quiz]
    hw[Assignment Panel]
    analytics[Course Aanalytics]

    home --> |Not Important for analysis| video
    home --> |OWN or ALL| analytics
    home --> |NOT Solution & Submission| quiz
    home --> |For URL of PDF| hw
```

## URL Assembly

1. Home Page
   - Require proper SignIn from browser.
   - https://learn.pwskills.com/lesson/Working-with-MongoDB/63ef8452ff4766650ddc4e5d/course/Data-Science-masters/63a2eb428899436daf7eb489
   - https://learn.pwskills.com/course/Data-Science-masters/63a2eb428899436daf7eb489
2. Videos - Skip
3. Analytics
   1. Using `requests` module you can get **all** analytics.
      - https://learn.pwskills.com/course-analytics/Data-Science-masters/63a2eb428899436daf7eb489
   2. Get your own analytics using `cURL` command.
      - https://api.pwskills.com/v1/learn/analytics/progress/course/63a2eb428899436daf7eb489
   3. Get you submission data using `cURL` command. Also, Submission data is fetches by the Home page too.
      - https://api.pwskills.com/v1/learn/submission/63a2eb428899436daf7eb489
4. Quiz
   1. Get a particular Quiz data. If qiuz is submitted, marks details includes in the response data.
      - https://api.pwskills.com/v1/learn/lesson/course/63a2eb428899436daf7eb489/63fa2330182c6727c14b9189
5. Assignment
   1. Get a particular Assignment data. If qiuz is submitted, marks details includes in the response data.
      - https://api.pwskills.com/v1/learn/lesson/course/63a2eb428899436daf7eb489/63fa1682182c67aa884b90ef
